# Supply Chain Management Dashboard

In [1]:
import pandas as pd
import zipfile
import os

In [2]:
# Step 1: Load the CSV file from the zip archive
zip_file_path = 'Supply Chain Analysis.zip'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()  # Extracts files into the current directory

# Load the CSV file into a pandas DataFrame
csv_file_path = 'supply_chain_data.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(df.head())
print(df.describe())
print(df.info())

  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626               Unknown             1          10   
3        7766.836426            Non-binary            23          13   
4        2686.505152            Non-binary             5           3   

   Order quantities  ...  Location Lead time  Production volumes  \
0                96  ...    Mumbai        29          

In [3]:
# Create Features for Supplier Performance, Delivery Times, and Inventory Levels

In [4]:
# Step 1: Create Supplier Performance Metrics
# Group by 'Supplier name' to calculate various supplier performance metrics
supplier_performance = df.groupby('Supplier name').agg({
    'Lead times': 'mean',                # Average lead time
    'Defect rates': 'mean',              # Average defect rate
    'Costs': 'sum',                      # Total costs per supplier
    'Production volumes': 'sum',         # Total production volumes
    'Revenue generated': 'sum',          # Total revenue per supplier
    'Manufacturing lead time': 'mean'    # Average manufacturing lead time
}).reset_index()

# Rename columns for clarity
supplier_performance.columns = [
    'Supplier', 'Avg_Lead_Times', 'Avg_Defect_Rates', 'Total_Costs', 
    'Total_Production_Volumes', 'Total_Revenue', 'Avg_Manufacturing_Lead_Time'
]

# Step 2: Delivery Times per Supplier
# Calculate average delivery time for each supplier
delivery_times_stats = df.groupby('Supplier name').agg({
    'Lead times': ['mean', 'std', 'min', 'max'],  # Delivery times statistics
    'Shipping times': ['mean', 'std', 'min', 'max']  # Shipping times statistics
}).reset_index()

# Flatten the multi-level column names
delivery_times_stats.columns = ['Supplier', 'Avg_Delivery_Time', 'Delivery_Time_Std', 'Min_Delivery_Time', 'Max_Delivery_Time',
                                'Avg_Shipping_Time', 'Shipping_Time_Std', 'Min_Shipping_Time', 'Max_Shipping_Time']

# Step 3: Inventory Levels (No date, so just summarizing inventory by supplier)
inventory_summary = df.groupby('Supplier name').agg({
    'Stock levels': 'mean',  # Average stock levels by supplier
    'Order quantities': 'sum',  # Total order quantities by supplier
    'Production volumes': 'sum'  # Total production volumes by supplier
}).reset_index()

# Rename columns for clarity
inventory_summary.columns = ['Supplier', 'Avg_Stock_Levels', 'Total_Order_Quantities', 'Total_Production_Volumes']

In [5]:
# Grouping Data for Analysis

In [6]:
# Grouping Inventory Levels by Supplier and Product Type
inventory_by_supplier = df.groupby(['Supplier name', 'Product type']).agg({
    'Stock levels': 'sum',  # Sum of stock levels per supplier and product type
    'Order quantities': 'sum',  # Sum of order quantities
    'Production volumes': 'sum'  # Sum of production volumes
}).reset_index()

# Delivery Times by Supplier (for box plot visualization)
# This is used for box plot in Tableau to show the distribution of delivery times
delivery_times = df[['Supplier name', 'Lead times']]

In [7]:
# Save inventory levels data
inventory_csv_file = 'inventory_by_supplier.csv'
inventory_by_supplier.to_csv(inventory_csv_file, index=False)

# Save supplier performance data
supplier_performance_csv_file = 'supplier_performance.csv'
supplier_performance.to_csv(supplier_performance_csv_file, index=False)

# Save delivery times data
delivery_times_csv_file = 'delivery_times.csv'
delivery_times.to_csv(delivery_times_csv_file, index=False)

print("Processed datasets saved for Tableau visualization:")
print(f"Inventory levels by supplier: {inventory_csv_file}")
print(f"Supplier performance: {supplier_performance_csv_file}")
print(f"Delivery times by supplier: {delivery_times_csv_file}")

Processed datasets saved for Tableau visualization:
Inventory levels by supplier: inventory_by_supplier.csv
Supplier performance: supplier_performance.csv
Delivery times by supplier: delivery_times.csv
